# web scraping RIP.ie with beautifulsoup

In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    if soup.title == None:
        title = ''
    else:        
        title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [9]:
get_dn_page(390045)

390045 2019-06-05 00:00:00 Margaret  Kelly Avenue Grove, Ballymodan Place, Bandon,  Cork Cork


('Margaret  Kelly',
 datetime.datetime(2019, 6, 5, 0, 0),
 'Cork',
 'Avenue Grove, Ballymodan Place, Bandon,  Cork',
 "Kelly (Avenue Grove, Ballymodan Place, Bandon and late of Currivreeda West) on June 5th 2019. Margaret, beloved daughter of the late Timothy and Ellen. Sadly missed by her loving sisters Ann and Eileen, brothers Teddy and John, Margaret's Partner William, brothers-in-law, sisters-in-law, nieces, nephews, relatives and good friend Malcolm.;;Rosary on Friday evening at 7pm in St. Patrick's Church, Bandon. Requiem Mass on Saturday at 12 noon, funeral afterwards to the adjoining cemetery.;;May Margaret Rest in Peace")

In [2]:
#df = pd.read_csv('rip_dn_scrape.csv')
df = pd.read_pickle('rip_dn_scrape.pkl')
len(df)

486800

## iterate over a range of ids and get info

In [ ]:
#read current table in so we skip those already done
df = pd.read_pickle('rip_dn_scrape.pkl')
print (len(df))
ids = list(df.id)

results={}
for n in range(486000,488515):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page(n)
    if name == '':
        continue
    results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

In [12]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()
res = res.rename(columns={'index':'id'})

<ipython-input-12-849ec239fce7>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()


In [13]:
res

,id,name,date,county,address,notice
0,487776,Jim (James) BYRNE,2022-02-27,Dublin,"late of Michael Mallin House, Vicar Street, Du...","BYRNE, Jim (James) - 27 February 2022 - (late..."
1,487777,Margaret Higgins (née Armstrong),2022-02-11,,"Castlecarra, Carnacon, Mayo","Margaret Higgins (nee Armstrong) Coventry, Eng..."
2,487778,Mary Spellissy-Killeen,2022-02-27,Clare,"Clonfeigh, Ennis, Clare","Mary Spellissy Killeen, Clonfeigh, Ennis and f..."
3,487779,Martina Hayes,2022-02-23,Cork,"Peacock Row, Cork City, Cork","HAYES (Cork) On 23rd February 2022, unexpected..."
4,487780,Anthony Doherty,2022-02-26,Dublin,"Kilbarrack, Dublin","Doherty, Anthony - February 26, 2022 (late of ..."
...,...,...,...,...,...,...
730,488510,Tony Stack,2022-03-05,Kerry,"Toor, Duagh, Kerry","Tony Stack, Toor, Duagh, Co. Kerry. Suddenly, ..."
731,488511,David Wilson,2022-03-06,Donegal,"Cully, Donegal Town, Donegal",Reposing at his daughter Rebecca’s residence. ...
732,488512,Maurice O'Shaughnessy,2022-03-06,Cork,"Late of Farranferris Park, Farranree, Cork / ...",O’SHAUGHNESSY (Cork) Late of Farranferris Park...
733,488513,Michael Sweeney,2022-02-13,Longford,"Brocklagh, Drumlish, Longford","Michael Sweeney (Ballyedmond, Clonberne, Co. G..."


In [14]:
new = pd.concat([df,res]).reset_index(drop=True)
new=new[~new.id.duplicated(keep='first')]
print (len(df),len(res),len(new))
new.to_pickle('rip_dn_scrape.pkl')

487713 735 488448


In [9]:
#x.to_csv('rip_dn_scrape.csv')

## clean data

In [15]:
x=new
print (len(x))
x=x.replace('',None).dropna(axis=0,subset=['date'])
x['date'] = pd.to_datetime(x['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
x=x.drop_duplicates(['name','address'])
x=x.drop_duplicates(['name','date','county'])
x = x[~x.address.isnull()]
nc = ['Fermanagh','Armagh','Tyrone','Down','Antrim','Derry']
x = x[~x.county.isin(nc)]
x = x[~x.address.str.contains('|'.join(nc))]
x=x.sort_values('id')
print (len(x))
#x.to_csv('rip_dn_scrape_processed.csv')
x.to_pickle('rip_dn_scrape_processed.pkl')

488448
453688
